In [1]:
import os
os.chdir('../src')

In [2]:
import datetime

import pandas as pd
import numpy as np
import rasterio
import cv2

from config import _C as cfg
from utils import create_tile_v2, to_mask, rle2mask
from models.loss import dice_coefficient

identity = rasterio.Affine(1, 0, 0, 0, 1, 0)

In [3]:
# Provo a fare un check con le tile create con la procedura

In [4]:
df = pd.read_csv('/home/giorgio/Scrivania/Kaggle/hubmap/data/train.csv')
valid_id = cfg.DATASET.VALID_ID[0]

res = create_tile_v2(
    valid_id,
    df,
    cfg
)

In [5]:
path_img = os.path.join(
    cfg.DATA_DIR, 'train', valid_id+'.tiff'
)
dataset = rasterio.open(path_img, transform=identity, num_threads = 'all_cpus')
h, w = dataset.shape

encoding = df[df['id']==valid_id]['encoding'].values[0]
mask = rle2mask(encoding, (w, h))

In [6]:
mask = cv2.resize(mask, dsize=None, fx=cfg.DATASET.IMG_SCALE, fy=cfg.DATASET.IMG_SCALE, interpolation = cv2.INTER_AREA)

In [7]:
h, w = mask.shape

In [8]:
mask_pred = to_mask(
    res['mask_tile'],
    res['coord'],
    h,
    w,
    cfg.DATASET.TRAIN_TILE_SIZE
)

In [9]:
dice_coefficient(mask, mask_pred)

0.994824446489055

In [16]:
from models.model import build_model
from data_builder.transforms import get_valid_transform
import albumentations as A
import torch

In [11]:
model = build_model(cfg)

In [12]:
ckpt = torch.load('/home/giorgio/Scrivania/Kaggle/hubmap/experiments/resnet34/2020-12-29/unet_best.ckpt')

In [13]:
model.load_state_dict(ckpt['model_state_dict'])
del ckpt

In [23]:
model = model.to(cfg.DEVICE)
model = model.eval()

In [33]:
tile_image = res['img_tile']
tile_image = np.stack(tile_image)[..., ::-1]
print(tile_image.shape)
tile_image = np.ascontiguousarray(tile_image.transpose(0,3,1,2))
print(tile_image.shape)

(695, 320, 320, 3)
(695, 3, 320, 320)


In [17]:
norm = A.Normalize()

In [21]:
tile_image2 = res['img_tile']
tile_image2 = np.stack(tile_image2)[..., ::-1]
tile_image2 = norm(image=tile_image2)['image']
tile_image2 = np.ascontiguousarray(tile_image2.transpose(0,3,1,2))

print(tile_image2.shape)

(695, 3, 320, 320)


In [26]:
batch = np.array_split(tile_image2, len(tile_image2)//4)
    
tile_prob = []
#itero per tutti i batch
for num, imgs in enumerate(batch):
    imgs = torch.from_numpy(imgs).to(cfg.DEVICE)
    with torch.no_grad():
        y_hat = model(imgs)
        prob = torch.sigmoid(y_hat)

        tile_prob.append(prob.detach().cpu().numpy())

In [29]:
 tile_probz = np.concatenate(tile_prob).squeeze()

(695, 320, 320)

In [32]:
mask_pred = to_mask(
    tile_probz,
    res['coord'],
    h,
    w,
    cfg.DATASET.TRAIN_TILE_SIZE
)

In [33]:
mask_pred.shape

(7610, 5560)

In [39]:
for thr in np.linspace(0, 1, 21):
    predict = (mask_pred>thr).astype(np.float32)
    print(thr, dice_coefficient(predict, mask))

0.0 0.08642292939687168
0.05 0.8464845730068935
0.1 0.8306207463036845
0.15000000000000002 0.8144497504865558
0.2 0.7982093015864905
0.25 0.7818526662721169
0.30000000000000004 0.764338096722404
0.35000000000000003 0.7451995578566447
0.4 0.7223447746454441
0.45 0.695123764528971
0.5 0.6640785442244219
0.55 0.6312984237242854
0.6000000000000001 0.5969490633189521
0.65 0.5586122325211261
0.7000000000000001 0.5126859682696463
0.75 0.4563277629925834
0.8 0.3895218832171774
0.8500000000000001 0.30914553294079467
0.9 0.20565976038798078
0.9500000000000001 0.06028429598562651
1.0 0.0
